#### 分类器任务和数据介绍

- 构造一个将不同图像进行分类的神经网络分类器，对输入的图片进行判别并完成分类

- 本案例采用CIFAR10数据集作为原始的图片数据

- CIFAR10数据集介绍:数据集中每张图片的尺寸是3x32x32代表彩色3通道

- CIFAR10数据集总共有10种不同的分类，分别是"airplane", "automobile". "bird", "cat", "deer". "dog", "frog", "horse", "ship", "truck"

#### 训练分类器的步骤

- 使用torchvision下载CIFAR10数据集
- 定义卷积神经网络
- 定义损失函数
- 在训练集上训练模型
- 在测试集上测试模型

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

##### 下载数据集并将图片进行调整，因为torchvision数据集的输出是PILImage格式，数据域在[0,1]，我们将其转换为标准数据域[-1,1]的张量格式

In [5]:
#torchvision数据集的输出是PILImage格式，数据域在[0,1]，我们将其转换为标准数据域[-1,1]的张量格式
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


##### 注意
- 如果你是在Windows系统下运行上述代码，并且出现报错信息"BrokenPipeError",可以尝试将torch.utils.data.DataLoader()中的num_workers设置为0

In [ ]:
# 展示若干训练集的图片
# 导入画图包和numpy
import matplotlib.pyplot as plt
import numpy as np

# 构建展示图片的函数
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# 从数据迭代器中读取一张图片
dataiter = iter(trainloader)
images, labels = dataiter.__next__()

# 展示图片
imshow(torchvision.utils.make_grid(images))
# 打印标签label
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

##### 定义卷积神经网络

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __int__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 变换x的形状以适配全连接层的输入
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

##### 3.定义损失函数
- 采用交叉熵损失函数和随机梯度下降优化器

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

##### 4.在训练集上训练模型
- 采用基于梯度下降的优化算法，都需要很多个轮次的迭代训练

In [15]:
for epoch in range(2):
    running_loss = 0.0
    # 按批次迭代训练模型
    for i, data in enumerate(trainloader, 0):
        # 从data中取出含有输入图像张量inputs，标签张量labels
        inputs, labels = data

        # 第一步:首先将优化器梯度归零
        optimizer.zero_grad()

        # 第二步: 输入图像张量进入网络中，得到输出张量outputs
        outputs = net(inputs)

        #  利用网络的输出outputs和标签labels计算损失值
        loss = criterion(outputs, labels)

        # 反向传播+参数更新，是标准代码的标准流程
        loss.backward()
        optimizer.step()

        # 打印轮次和损失值
        running_loss += loss.item()
        if (i+1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <DDABACEB-F2EA-368C-80DD-40745DFB96F8> /Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <2729F3D6-2C8E-3DA6-9E14-4070A60558E5> /Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/zhangli/anaconda3/envs

NameError: name 'optimizer' is not defined

#### 保存模型

In [16]:
# 首先设定模型的保存路径
PATH = './cifar_net.pth'
# 保存模型的状态字典
torch.save(net.state_dict(), PATH)

##### 5.在测试集上测试模型
- 第一步:展示测试集中的若干图片

In [17]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# 打印原始图片
imshow(torchvision.utils.make_grid(images))

# 打印真实标签
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <DDABACEB-F2EA-368C-80DD-40745DFB96F8> /Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <2729F3D6-2C8E-3DA6-9E14-4070A60558E5> /Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/zhangli/anaconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/zhangli/anaconda3/envs

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

##### 加载模型并对测试图片进行预测

In [ ]:
# 首先实例化模型的类对象
net = Net()
# 加载训练阶段保存好的模型状态字典
net.load_state_dict(torch.load(PATH))

# 利用模型对图片进行预测
outputs = net(images)

# 共有10个类别，采用模型计算出的概率最大的作为预测的类别
_, predicted = torch.max(outputs, 1)

# 打印预测标签的结果
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# 全部测试集上的表现
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' %(100 * correct / total))

##### 为了更加细致的看一下模型在哪些类别上表现的更好，在哪些类别上表现的更差，我们分类别进行准确率计算

In [18]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
print(class_correct)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s: %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


#### 当训练模型的时候，只需要将模型转移到GPU上，同时将输入的图片和标签页转移到GPU上即可

In [ ]:
# 将模型转移到GPU上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

# 将输入的图片张量和标签张量转移到GPU上
inputs, labels = data[0].to(device), data[1].to(device)